# SMS Spam Classification
This notebook illustrates classification of SMS as SPAM or NOT SPAM using Recurrent Neural Network and LSTM.

In [1]:
from collections import Counter
import tensorflow as tf
%matplotlib inline
import matplotlib.pyplot as plt
import csv
import pandas
import sklearn
import pickle
from wordcloud import WordCloud
import pandas as pd
import numpy as np
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.tree import DecisionTreeClassifier 
from sklearn.model_selection import learning_curve    
import joblib

C:\Users\ITD-STU\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\ITD-STU\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\ITD-STU\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\ITD-STU\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:519: FutureWarnin

In [2]:
data = pd.read_csv('data/emails.csv',encoding='latin-1')
data.head(10)

,text,label
0,Subject: naturally irresistible your corporate...,1
1,Subject: the stock trading gunslinger fanny i...,1
2,Subject: unbelievable new homes made easy im ...,1
3,Subject: 4 color printing special request add...,1
4,"Subject: do not have money , get software cds ...",1
5,"Subject: great nnews hello , welcome to medzo...",1
6,Subject: here ' s a hot play in motion homela...,1
7,Subject: save your money buy getting this thin...,1
8,Subject: undeliverable : home based business f...,1
9,Subject: save your money buy getting this thin...,1


## Data Preprocessing
Lets save our labels and messages to text files

Removing Stopwords from the messages

In [ ]:
import string
def text_process(text):
    text = text.translate(str.maketrans('', '', string.punctuation))
    text = [word for word in text.split() if word.lower() not in stopwords.words('english')]
    
    return " ".join(text)

In [ ]:
data['text'] = data['text'].apply(text_process)

In [ ]:
np.savetxt(r'data\messages.txt', data['text'].values, fmt='%s')
np.savetxt(r'data\labels.txt', data['label'].values, fmt='%s')

In [4]:
with open('data/messages.txt', encoding="ISO-8859-1") as f:
    messages = f.read()
with open('data/labels.txt',encoding="ISO-8859-1") as f:
    labels = f.read()

In [5]:
data['label'].value_counts()

0    4358
1    1368
Name: label, dtype: int64

### Remove punctuations such a (. , !) etc and seperate using delimiter

In [14]:
from string import punctuation
all_text = ''.join([c for c in messages if c not in punctuation])
messages = all_text.split('\n')

all_text = ' '.join(messages)
words = all_text.split()

In [15]:
print (all_text[:500])
print ("\n")
print (words[:20])

Subject naturally irresistible your corporate identity  lt is really hard to recollect a company  the  market is full of suqgestions and the information isoverwhelminq  but a good  catchy logo  stylish statlonery and outstanding website  will make the task much easier   we do not promise that havinq ordered a iogo your  company will automaticaily become a world ieader  it isguite ciear that  without good products  effective business organization and practicable aim it  will be hotat nowadays mar


['Subject', 'naturally', 'irresistible', 'your', 'corporate', 'identity', 'lt', 'is', 'really', 'hard', 'to', 'recollect', 'a', 'company', 'the', 'market', 'is', 'full', 'of', 'suqgestions']


### Building our vocabulary and converting messages to vectors

In [42]:
split_words = Counter(words)
sorted_split_words = sorted(split_words, key=split_words.get, reverse=True)
vocab_to_int = {c : i for i, c in enumerate(sorted_split_words,1)}

# Convert the reviews to integers, same shape as reviews list, but with integers
messages_ints = []
for message in messages:
    messages_ints.append([vocab_to_int[i] for i in message.split()])

In [43]:
print (sorted_split_words[:50])
print ("\n")
print (messages[0])
print (messages_ints[0])
print ("\n")
print (len(messages[0]))
print (len(messages_ints[0]))

['the', 'to', 'and', 'of', 'a', 'you', 'in', 'i', 'for', 'enron', 'on', 'is', 'ect', 'this', 'your', 'be', 'that', 'with', 'we', 'vince', 'will', 'have', 'at', 'from', 'it', 'are', 's', 'as', 'Subject', 'hou', 'com', 'by', 'or', 'if', 'am', 'please', '2000', 'kaminski', 'not', 'subject', 'me', 'would', 'our', 'can', 're', 'cc', 'j', 'my', 'an', '1']


Subject naturally irresistible your corporate identity  lt is really hard to recollect a company  the  market is full of suqgestions and the information isoverwhelminq  but a good  catchy logo  stylish statlonery and outstanding website  will make the task much easier   we do not promise that havinq ordered a iogo your  company will automaticaily become a world ieader  it isguite ciear that  without good products  effective business organization and practicable aim it  will be hotat nowadays market  but we do promise that your marketing efforts  will become much more effective  here is the list of clear  benefits  creativeness  hand  made

#### Converting labels to 0 and 1 - SPAM:1 and NOT SPAM:0 

In [44]:
labels = labels.split("\n")
labels = np.array([0 if label == "0" else 1 for label in labels])

In [45]:
print(labels)

[1 1 1 ... 0 0 1]


In [46]:
from collections import Counter

message_lens = Counter([len(x) for x in messages_ints])
print("Zero-length messages: {}".format(message_lens[0]))
print("Maximum message length: {}".format(max(message_lens)))

Zero-length messages: 1
Maximum message length: 5003


In [47]:
messages_ints = [message for message in messages_ints if (len(message)>0)]

### Padding vectors with zeros so that all inputs are of same length

In [48]:
seq_len = 200
num_messages = len(messages)
features = np.zeros([num_messages, seq_len], dtype=int)
for i, row in enumerate(messages_ints):
    features[i, -len(row):] = np.array(row)[:seq_len]

In [50]:
features[1]

array([    0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,     0,     0,     0,     0,     0,     0,
           0,     0,     0,    29,     1,   503,   194, 22179, 22180,
          12,  4890,    76, 22181,    39, 16808, 22182,     3, 22183,
          73, 22184,

### Splitting into training, validation and test data

In [51]:
split_frac1 = 0.8

idx1 = int(len(features) * split_frac1)
train_x, val_x = features[:idx1], features[idx1:]
train_y, val_y = labels[:idx1], labels[idx1:]

split_frac2 = 0.5
idx2 = int(len(val_x) * split_frac2)
val_x, test_x = val_x[:idx2], val_x[idx2:]
val_y, test_y = val_y[:idx2], val_y[idx2:]

print("\t\t\tFeature Shapes:")
print("Train set: \t\t{}".format(train_x.shape), 
      "\nValidation set: \t{}".format(val_x.shape),
      "\nTest set: \t\t{}".format(test_x.shape))

print("\t\t\Label Shapes:")
print("Train set: \t\t{}".format(train_y.shape), 
      "\nValidation set: \t{}".format(val_y.shape),
      "\nTest set: \t\t{}".format(test_y.shape))

			Feature Shapes:
Train set: 		(4581, 200) 
Validation set: 	(573, 200) 
Test set: 		(573, 200)
		\Label Shapes:
Train set: 		(4581,) 
Validation set: 	(573,) 
Test set: 		(573,)


### Initial prediction using Logistic Regression

In [52]:
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score

clf = LogisticRegression()
clf.fit(train_x,train_y)
p = clf.predict(val_x)
print (accuracy_score(val_y,p))

0.7260034904013961


C:\Users\ITD-STU\Anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
